# Наполнение базы данных.

Данные для наполнения берутся с сайта [викиконспектов](https://neerc.ifmo.ru/wiki/index.php?title=%D0%97%D0%B0%D0%B3%D0%BB%D0%B0%D0%B2%D0%BD%D0%B0%D1%8F_%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B8%D1%86%D0%B0) от университета ИТМО. Скачиваемые данные очищаются от HTML-тегов, пишутся в текстовые файлы по темам, которые кладутся в директорию `text_db`.

In [86]:
%pip install requests beautifulsoup4 mwparserfromhell wikitextparser pywikibot

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.comNote: you may need to restart the kernel to use updated packages.




[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: C:\Users\savch\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [87]:
from bs4 import BeautifulSoup
import mwparserfromhell
import wikitextparser
from pywikibot import textlib
import re

def get_text(node, trim=False):
    if node is None:
        return ""
    res = node.find_all(r"\<[/\w]+\>", recursive=False)
    if res:
        if trim:
            res = list(res)[1:-1]
        return '\n'.join(get_text(part) for part in res)
    else:
        return node.get_text()
    
def extract(html):
    soup = BeautifulSoup(html, "html.parser")
    art = soup.find("article")
    return get_text(html, True)

def clean_wikitext(wikitext):
    text = textlib.removeHTMLParts(wikitext, keeptags=[])
    text.replace("\{\{", "").replace("\}\}", "").replace("|", "")
    return text

In [88]:
import requests

url = "https://neerc.itmo.ru/wiki/api.php"

def get_web_content(title):
    parameters = {
       "action": "query",
        "prop": "revisions",
        "titles": title,
        "rvprop": "content",
        "format": "json"
    }

    response = requests.get(url, params=parameters)
    data = response.json()
    pages = data['query']['pages']
    for page_id, page_info in pages.items():
        if page_id != "-1": 
            return page_info['revisions'][0]['*']
        else:
            return None

In [89]:
def write_to_file(conspects, filename):
    with open(filename, "w", encoding="utf-8") as conspects_file:
        for title in conspects:
            conspects_file.write(conspects[title])
            conspects_file.write('-----')

In [90]:
def get_content(titles):
    content = {
        title: "" for title in titles
    }
    for title in titles:
        content[title] = str(clean_wikitext(get_web_content(title)))

    return content

In [93]:
titles = [
    "Основные_определения,_связанные_со_строками",
    "Регулярные_языки:_два_определения_и_их_эквивалентность",
    "Детерминированные_конечные_автоматы",
    "Прямое_произведение_ДКА",
    "Преобразование_регулярного_выражения_в_ДКА",
    "Недетерминированные_конечные_автоматы",
    "Построение_по_НКА_эквивалентного_ДКА,_алгоритм_Томпсона",
    "Автоматы_с_eps-переходами._Eps-замыкание",
    "Эквивалентность_состояний_ДКА",
    "Минимизация_ДКА,_алгоритм_Хопкрофта_(сложность_O(n_log_n))",  
    "Доказательство_нерегулярности_языков:_лемма_о_разрастании",
    "Решение_уравнений_в_регулярных_выражениях",
    "Формальные_грамматики",
    "Иерархия_Хомского_формальных_грамматик",
    "Неукорачивающие_и_контекстно-зависимые_грамматики,_эквивалентность",
    "Правоконтекстные_грамматики,_эквивалентность_автоматам"
]

In [94]:
content = get_content(titles)
write_to_file(content, "text_db/formal_languages.txt")